In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [119]:
def getTitles(end_page): 
    result = {}
    for i in range(1, end_page):
        url = "https://westlafayette.evergreenindiana.org/Search/Results?lookfor=&searchIndex=Keyword&filter[]=format_category%3A%22Books%22&filter[]=availability_toggle%3A%22local%22&sort=relevance&view=list&searchSource=local&page=" + str(i)
        response = requests.get(url)
        response = response.content
        soup = BeautifulSoup(response, 'html.parser')
        all_books_on_page = soup.find('div', id="searchInfo")
        books = all_books_on_page.find_all('h3')
        for book in books:
            title = book.find('a')
            name = title.attrs['aria-label']
            link = title.attrs['href']
            result[name] = "https://westlafayette.evergreenindiana.org" + link
    return result

In [120]:
test_descs = getTitles(2)

In [121]:
def getDescriptions(url_list):
    options = webdriver.ChromeOptions()
    # Launch Chrome:
    driver = webdriver.Chrome(options=options)
    result = []
    for i in url_list:
        driver.get(str(url_list[i]))
        try:
            WebDriverWait(driver, 10).until_not(
                EC.text_to_be_present_in_element(
                    (By.ID, 'descriptionPlaceholder'),
                    'Loading Description...'
                )
            )
        except TimeoutException:
            print(f"Timeout while waiting for description to load on {str(url_list[i])}")
            continue
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        dynamic_content = soup.find('div', id='descriptionPlaceholder')
        result.append(dynamic_content.get_text(strip=True))
    driver.quit()
    return result

In [122]:
getDescriptions(test_descs)

['Describes the life and illustrious career of the writer, scientist, and statesman, from his youth in Boston to his years as a printer, his diplomatic visits abroad, and his role in the drafting of the U.S. Constitution.',
 'Description Not Provided',
 'The instantNew York Timesbestseller!\xa0“Warm and perceptive.”—New York Times“Griffin Dunne knows how to tell a story."—Washington Post"Dunne is a prospector for the incandescent detail.”—Los Angeles Times“What a remarkable and moving story filled with twists and turns, the most famous of faces, and a complex family revealed with loving candor. I was blown away by Griffin Dunne’s life and his ability to capture so much of it in these beautifully written pages.”—Anderson CooperGriffin Dunne’s memoir of growing up among larger-than-life characters in Hollywood and Manhattan finds wicked humor and glimmers of light in even the most painful of circumstancesAt eight, Sean Connery saved him from drowning. At thirteen, desperate to hook up wi